#### Introduction\n",
    "I am considering the numerical problem which approximates the dispersion of a passive pollutant in a groundwater flow field via diffusive and advective processes, all other processes left out of consideration. 'Passive pollutant' in this contexts refers to a pollutant which does not affect the flow field in which it is dispersed, and so interacts with the problem in a passive sense. Numerical methods to model groundwater flow are key to understanding one of our most important resources (i.e. freshwater) and provide insight into how to manage and protect vital watersheds. Numerical models can, for example, give us insight into the how changes in precipitation may affect groundwater drinking sources for communities, or how a leak or spill of contaminants will be absorbed and travel within an aquifer. The problem that I have chosen to investigate physically resembles the problem of how a passive pollutant is transported in a simple time and space invariant flow field via advective and diffusive processes.\n",
    "\n",
    "\n",
    "#### Derivation of the Problem: \n",
    "\n",
    "\n",
    "We can start from a 2-D convection-diffusion equation that includes a source term:\n",
    "\n",
    "\n",
    "$$ \\frac{\\partial C}{\\partial t}=D\\left(\\frac{\\partial^{2} C}{\\partial x^{2}}+\\frac{\\partial^{2} C}{\\partial y^{2}}\\right)-\\left(\\frac{\\partial u}{\\partial x} C+u \\frac{\\partial C}{\\partial x}\\right)-\\left(\\frac{\\partial v}{\\partial y} C+v \\frac{\\partial C}{\\partial y}\\right) + Q(x,y,t) $$  \n",
    "\n",
    "Where we can say that $C = C(x,y,t)$ is the pollutant concentration at a specified time and space, and $D$ is a diffusion constant. We can also define a velocity field $\\vec{velocity} = (u,v)$. Working from left to right, each term represents accumulation, diffusion in $x, y$ down-gradient from high to low concentrations, and advection in $x, y$ along the flow field, respectively. \n",
    "\n",
    "\n",
    "\n",
    "#### Discretization of the Continuity Equation: \n",
    "\n",
    "\n",
    "We can discretize the problem in a simple forward time centered space (FTCS) scheme as follows:\n",
    "\n",
    "\n",
    "$$ \\frac{C_{i, j}^{n+1}-C_{i, j}^{n}}{\\Delta t}=D \\frac{C_{i-1, j}^{n}-2 C_{i, j}^{n}+C_{i+1, j}^{n}}{\\Delta x^{2}}+D \\frac{C_{i, j-1}^{n}-2 C_{i, j}^{n}+C_{i, j+1}^{n}}{\\Delta y^{2}} - \\left(\\frac{u_{i+1, j}^{n}-u_{i-1, j}^{n}}{2 \\Delta x} C_{i, j}^{n}+u_{i, j}^{n} \\frac{C_{i+1, j}^{n}-C_{i-1, j}^{n}}{2 \\Delta x}\\right)-\\left(\\frac{v_{i, j+1}^{n}-v_{i, j-1}^{n}}{2 \\Delta y} C_{i, j}^{n}+ v_{i, j}^{n}\\frac{C_{i, j+1}^{n}-C_{i, j-1}^{n}}{2 \\Delta y}\\right) + Q^{n}_{i,j}$$  \n",
    "\n",
    "Which has accuracy on the order of: $\\mathcal{O}\\left(\\Delta t,(\\Delta x)^{2},(\\Delta y)^{2}\\right)$\n",
    "\n",
    "Rearranging to solve for $T_{i,j}^{n+1}$ explicitly.:  \n",
    "$$ T_{i, j}^{n+1}=T_{i, j}^{n}+\\Delta t \\left( \\kappa \\frac{T_{i-1, j}^{n}-2 T_{i, j}^{n}+T_{i+1, j}^{n}}{\\Delta x^{2}}+\\kappa \\frac{T_{i, j-1}^{n}-2 T_{i, j}^{n}+T_{i, j+1}^{n}}{\\Delta y^{2}} - \\left(\\frac{u_{i+1, j}^{n}-u_{i-1, j}^{n}}{2 \\Delta x} T_{i, j}^{n}+u_{i, j}^{n} \\frac{T_{i+1, j}^{n}-T_{i-1, j}^{n}}{2 \\Delta x}\\right)-\\left(\\frac{v_{i, j+1}^{n}-v_{i, j-1}^{n}}{2 \\Delta y} T_{i, j}^{n}+v_{i, j}^{n} \\frac{T_{i, j+1}^{n}-T_{i, j-1}^{n}}{2 \\Delta y}\\right) + Q^{n}_{i,j}\\right) $$\n",
    "\n",
    "If velocity field unchanging in $x, y, t$, then we can drop the $n, i, j$ superscripts on $ u, v $ and our differential terms for $ u, v $ vanish. \n",
    "\n",
    "\n",
    "\n",
    "For stability considerations, I wasn't able to derive any any stability conditions (e.g. CFL) or find any literature on what the stability of a FTCS scheme for a 2-D advection-diffusion might be. Without a handrail for where the model behaves in a stable way, I've used a relatively coarse spatial grid in comparison with my timestep. My understanding is that the timestep has an upper limit which is dependent on the spatial step, diffusivity, and velocity field, but I was unable to derive that relationship. \n",
    "\n",
    "With the general unstability of the FTCS method in mind, I hoped to implement an upwind scheme such as the Crank Nicholson scheme or a Duport-Frankl scheme.\n",
    "\n",
    "\n",
    "#### Crank-Nicholson Scheme:\n",
    "Assuming a spatially homogeneous flow field, our differential terms for $u,v$ disappear. We can rewrite our  $u \\frac{\\partial C}{\\partial x}$ and $v \\frac{\\partial C}{\\partial y}$ terms as an average between the $n$ and $n+1$ timesteps as follows: \n",
    "\n",
    "$$ u \\frac{\\partial C}{\\partial x} \\rightarrow u\\left(\\frac{1}{2}\\left(\\frac{C_{i+1,j}^{n+1}-C_{i-1,j}^{n+1}}{2 \\Delta x}+\\frac{C_{i+1,j}^{n}-C_{i-1,j}^{n}}{2 \\Delta x}\\right)\\right)$$\n",
    "\n",
    "$$ v \\frac{\\partial C}{\\partial y} \\rightarrow v\\left(\\frac{1}{2}\\left(\\frac{C_{i,j+1}^{n+1}-C_{i,j-1}^{n+1}}{2 \\Delta y}+\\frac{C_{i,j+1}^{n}-C_{i,j-1}^{n}}{2 \\Delta y}\\right)\\right)$$\n",
    "\n",
    "And so the Crank-Nicholson scheme can be written as follows: \n",
    "\n",
    "$$ \\frac{C_{i, j}^{n+1}-C_{i, j}^{n}}{\\Delta t}=D \\frac{C_{i-1, j}^{n}-2 C_{i, j}^{n}+C_{i+1, j}^{n}}{\\Delta x^{2}}+D \\frac{C_{i, j-1}^{n}-2 C_{i, j}^{n}+C_{i, j+1}^{n}}{\\Delta y^{2}} - \\left(\\frac{u}{2}\\left(\\frac{C_{i+1,j}^{n+1}-C_{i-1,j}^{n+1}}{2 \\Delta x}+\\frac{C_{i+1,j}^{n}-C_{i-1,j}^{n}}{2 \\Delta x}\\right)\\right)-\\left(\\frac{v}{2}\\left(\\frac{C_{i,j+1}^{n+1}-C_{i,j-1}^{n+1}}{2 \\Delta y}+\\frac{C_{i,j+1}^{n}-C_{i,j-1}^{n}}{2 \\Delta y}\\right)\\right)$$  \n",
    "\n",
    "\n",
    "Solving the implicit problem requires a linear algebra scheme which I unfortunately \n",
    "wasn't able to implement for this project. \n",
    "\n",
    "#####  * Please run the script below#### "